In [2]:
import jieba
import numpy as np
import os, glob, time, copy, random, zipfile
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F
import torchvision
from torchvision import models, transforms

In [3]:
from os import listdir
from os.path import isfile, join,splitext
t_dir = 'dataTrainComplete'
txt_fnames = [splitext(f)[0] for f in listdir(t_dir) if isfile(join(t_dir, f))] #Article filenames

In [6]:
vector_dict = [{},{},{}]
crop_list = open('Keywords/02crop.list.csv', "r",encoding='UTF-8-sig')
crop = crop_list.read()
crop_line_sep = crop.splitlines()
print(len(crop_line_sep))
for idx,line in enumerate(crop_line_sep):
    l = line.split(',')
    for word in l:
        if(word == ''):continue
        jieba.add_word(word)
        vector_dict[0][word] = idx
pest_list = open('Keywords/02pest.list.csv', "r",encoding='UTF-8-sig')
pest = pest_list.read()
pest_line_sep = pest.splitlines()
print(len(pest_line_sep))
for idx,line in enumerate(pest_line_sep):
    l = line.split(',')
    for word in l:
        if(word == ''):continue
        jieba.add_word(word)
        vector_dict[1][word] = idx

chem_list = open('Keywords/02chem.list.csv', "r",encoding='UTF-8-sig')
chem = chem_list.read()
chem_line_sep = chem.splitlines()
print(len(chem_line_sep))
for idx,line in enumerate(chem_line_sep):
    l = line.split(',')
    for word in l:
        if(word == ''):continue
        jieba.add_word(word)
        vector_dict[2][word] = idx
#Keywords split by lines, keyword with more than one entry will be on the same line
print(len(vector_dict[0]),len(vector_dict[1]),len(vector_dict[2]))

188
207
369
216 282 864


In [10]:
vectors = {}
#Vector table with filename as key and vector as value
for fname in txt_fnames:
    txt = open(t_dir+'/'+fname+'.txt', "r",encoding="utf-8")
    content = txt.read()
    seg_list = jieba.cut(content, cut_all=True)
    vectors[fname] = [[False]*188,[False]*207,[False]*369]
    #Initialize an 0 vector for each file
    for seg in seg_list:
        if(seg in vector_dict[0]):
            vectors[fname][0][vector_dict[0][seg]] = True
        if(seg in vector_dict[1]):
            vectors[fname][1][vector_dict[1][seg]] = True
        if(seg in vector_dict[2]):
            vectors[fname][2][vector_dict[2][seg]] = True
            #Count keyword appearence - Bag of words

for key in vectors:
    v = vectors[key]
    for i in range(3):
        curr = 0
        for b in v[i]:
            if(b):curr = (curr << 1) + 1
            else:curr = curr << 1
        vectors[key][i] = curr

[98079714615416886934934209737619787751599303819750539264, 51338600619007276755004625409535357651227760593150672896, 0]


In [45]:
def countSetBits(n):
    count = 0
    while (n):
        n &= (n-1)
        count+= 1
    return count

In [114]:
out = []
for Ref in txt_fnames:
    for Test in txt_fnames:
        if(Ref == Test):continue
        if((vectors[Test][0] == 0 and vectors[Test][1] == 0) or (vectors[Ref][0] == 0 and vectors[Ref][1] == 0)):continue
        if(vectors[Test][0] ^ (vectors[Test][0] & vectors[Ref][0]) == 0 and vectors[Test][1] ^ (vectors[Test][1] & vectors[Ref][1]) == 0 and vectors[Test][2] ^ (vectors[Test][2] & vectors[Ref][2]) == 0):
            out.append((Test,Ref))

In [115]:
corr_list = open('TrainLabel.csv', "r",encoding='UTF-8-sig')
corr = corr_list.read()
corr_line_sep = corr.splitlines()
pos_labels = set() #faster lookup to filter out positive pairs, not used elsewhere
#All the associated article pairs given by train label
for line in corr_line_sep[1:]:
    l = line.split(',')
    pos_labels.add((l[0],l[1]))

In [116]:
count = 0
for o in out:
    if(o in pos_labels):count+=1
print(count,len(out),len(pos_labels))

996 2205 1383


In [119]:
import csv
from natsort import natsorted
print(len(out))
out = natsorted(out,key = lambda x:x[0])
with open('val_super_simple_TEST.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(["Test"]+["Reference"])
    for row in out:  
        spamwriter.writerow(row)

2205


In [135]:
Ref = '415'
Test = '3'
#print(countSetBits(vectors[Test][0] ^ (vectors[Test][0] & vectors[Ref][0])),
#countSetBits(vectors[Test][1] ^ (vectors[Test][1] & vectors[Ref][1])),
#countSetBits(vectors[Test][2] ^ (vectors[Test][2] & vectors[Ref][2])))
txt = open(t_dir+'/'+Ref+'.txt', "r",encoding="utf-8")
content = txt.read()
print(content)
seg_list = jieba.cut(content, cut_all=True)
for seg in seg_list:
    if(seg in vector_dict[0] or seg in vector_dict[1]):print(seg)
print("----")
txt = open(t_dir+'/'+Test+'.txt', "r",encoding="utf-8")
content = txt.read()
print(content)
seg_list = jieba.cut(content, cut_all=True)
for seg in seg_list:
    if(seg in vector_dict[0] or seg in vector_dict[1]):print(seg)

夜蛾拉警報，白粉等著來，臺南區農改場籲請加強防治。臺南區農改場、防檢局及田邊好幫手關心您。
目前氣候乾燥及正值季節轉換期間，田間夜蛾類害蟲密度逐漸上升，尤以落花生田區發生嚴重。同時天氣漸轉冷涼，早晚溫差變大，白粉病發生機率提高，為確保作物正常生長及良好收成，臺南區農業改良場籲請農友注意夜蛾類害蟲及白粉病防治。常見危害夜蛾有斜紋夜蛾及甜菜夜蛾，兩者幼蟲皆晝伏夜出，雜食性，可危害作物如葉菜類、茼蒿、落花生、田菁、毛豆、豌豆、大蒜、青蔥、甘藷、玉米、蓮花及花卉等數十種作物。幼蟲食量極大，會大量啃食植株心梢及葉片，阻礙作物的正常生長，影響產量及品質，嚴重時常導致全園廢耕。白粉病是常見的作物葉部病害，感染初期通常由下位葉開始發生，葉表面出現白至淺灰之圓形小病斑，擴大癒合後即好似一層糖霜覆蓋整個葉面，影響光合作用，若遇久旱不雨的天氣，嚴重時葉片黃化，提早落葉。於春、秋兩季較乾燥及光線不足的環境下危害劇烈，由於白粉病亦會感染葉柄、嫩蔓等部位，如病害繼續進行，最終作物全株表面皆覆滿白色粉狀物而呈青白色。
夜蛾類害蟲防治上應注意下列事項：(1)田間如發現夜蛾類害蟲卵塊、幼蟲，應立即捕殺或摘除(2)幼蟲及成蟲大多於太陽下山或傍晚時分開始活動，傍晚噴藥效果較佳，噴灑時應注意使用充足水量(3)參考植物保護手冊推薦用藥，選擇藥劑交互使用，以避免害蟲產生抗藥性(4)利用性費洛蒙長期誘殺雄蛾，以降低其田間密度。每公頃設5~10個點，放置高度約為離地150公分或作物生長點上方30~60公分，每月更新誘餌一次。作物種植密集地區應採共同防治，成效更佳(5)注意清園之工作，間作之作物應同時施藥，徹底清除蟲源(6)旱田作物於種植前整地翻犁後浸水一天以上，可將土中的蛹或幼蟲淹死(7)收穫前注意化學藥劑的使用，或改用蘇力菌防治，注意安全採收期，以免造成農藥殘留過量之問題(8)受害嚴重之綠肥作物或廢耕田區應儘速翻犁淹水，避免夜蛾類滋生蔓延。白粉病防治法如下：(1)保持良好通風狀況，避免莖葉過於茂密。(2)發病初期可施用葵無露或窄域油。(3)參考作物登記藥劑，由於白粉病病原菌易產生抗藥性，因此應選擇多種藥劑，輪流施用，以避免抗藥性產生，並遵守安全採收期。白粉病葉面葉背都會發生，因此防治時應注意均勻噴施藥劑於葉片兩面。藥劑防治請參考植物保護手冊登記藥劑，可於本場網站連結各項作物的相關用藥資料：http://www.

In [130]:
vector_dict[0]

{'文旦柚': 0,
 '文旦': 0,
 '麻豆文旦': 0,
 '水稻': 1,
 '青蔥': 2,
 '蔥': 2,
 '龍鬚菜': 3,
 '韭菜': 4,
 '甘藍': 5,
 '西瓜': 6,
 '芋頭': 7,
 '香蕉': 8,
 '甘藷': 9,
 '玉米': 10,
 '胡瓜': 11,
 '絲瓜': 12,
 '洋香瓜': 13,
 '菠菜': 14,
 '萵苣': 15,
 '十字花科': 16,
 '豆科': 17,
 '大蒜': 18,
 '茼蒿': 19,
 '落花生': 20,
 '花生': 20,
 '蔬菜': 21,
 '田菁': 22,
 '毛豆': 23,
 '豌豆': 24,
 '蓮花': 25,
 '梨': 26,
 '中國梨': 26,
 '高接梨': 26,
 '荔枝': 27,
 '玉荷包荔枝': 27,
 '龍眼': 28,
 '蓮霧': 29,
 '欒樹': 30,
 '台灣欒樹': 30,
 '果樹': 31,
 '草莓': 32,
 '草莓苗期': 32,
 '柑橘': 33,
 '紅棗': 34,
 '柿子': 35,
 '甜柿': 35,
 '桃子': 36,
 '桃': 36,
 '李子': 37,
 '枇杷': 38,
 '楊桃': 39,
 '檬果': 40,
 '芒果': 40,
 '木瓜': 41,
 '番石榴': 42,
 '芭樂': 42,
 '檸檬': 43,
 '印度棗': 44,
 '番茄': 45,
 '小果番茄': 45,
 '橙蜜香番茄': 45,
 '橙果小番茄': 45,
 '馬鈴薯': 46,
 '紅龍果': 47,
 '棗': 48,
 '蜜棗': 49,
 '瓜果類': 50,
 '豆菜類': 51,
 '洋蔥': 52,
 '豆類': 53,
 '龍柏': 54,
 '棗子': 55,
 '高粱': 56,
 '大豆': 57,
 '香瓜': 58,
 '南瓜': 59,
 '橄欖': 60,
 '葡萄': 61,
 '洋桔梗': 62,
 '柑桔': 63,
 '百香果': 64,
 '番荔枝': 65,
 '鳳梨釋迦': 66,
 '釋迦': 66,
 '樹豆': 67,
 '桶柑': 68,
 '茂谷柑': 69,
 '臍橙': 70,
 '晚崙西亞': 71,